In [104]:
import pandas as pd

# Récupération des données

## Jeu de données de base

In [156]:
fichiersource = "../../datasources/films/allocine_bigfoot_avis.csv"
src = pd.read_csv(fichiersource)
src.head()

,Note,Description,key
0,"3,0","Suite de ""Bigfoot Junior"" sorti en 2017, revoi...",0
1,"3,0",malgré un scénario moyen ce film d animation ...,1
2,"3,0",Un debut laborieux qui se veut dans l'air du t...,2
3,"3,5","Film vu au cinéma. \nTout d'abord, les graphis...",3
4,"5,0",Nous avons adoré avec mon fils. Je trouve que ...,4


In [164]:
src['Description'][1]

'malgré un scénario moyen ce film  d animation se regarde bien même.si.la mise en route est difficile et que tout le début soit dans.la.bande annonce.\nle film est visuellement très beau également'

## Création du vecteur de résultat (classe binaire)

In [165]:
def setClassBin(i):
    if (float(i.replace(',', '.')) > 3):
        return 1
    else:
        return 0

yList = [setClassBin(x) for x in src.Note]
y = pd.DataFrame(yList)

In [166]:
y.head()

,0
0,0
1,0
2,0
3,1
4,1


## Jeu de données pour l'apprentissage

In [167]:
X = src.drop('Note', axis=1)
X.head()

,Description,key
0,"Suite de ""Bigfoot Junior"" sorti en 2017, revoi...",0
1,malgré un scénario moyen ce film d animation ...,1
2,Un debut laborieux qui se veut dans l'air du t...,2
3,"Film vu au cinéma. \nTout d'abord, les graphis...",3
4,Nous avons adoré avec mon fils. Je trouve que ...,4


# Préparation des données

### Retire les éléments de ponctuation ...

In [168]:
import re

REMPLACE_SANS_ESPACE = re.compile("[;:!\'?,\"()\[\]]")
REMPLACE_AVEC_ESPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|[.]")
PUR_NOMBRE = re.compile("[0-9]")

def preprocess_reviews(reviews):
    reviews = [PUR_NOMBRE.sub("", line.lower()) for line in reviews] # retire les nomre (comme les années)
    reviews = [line.replace('\n', ' ')  for line in reviews] # Retire les \n (retours chariots)
    reviews = [REMPLACE_SANS_ESPACE.sub("", line.lower()) for line in reviews]
    reviews = [REMPLACE_AVEC_ESPACE.sub(" ", line) for line in reviews]
    return reviews

X['Description'] = pd.DataFrame(preprocess_reviews(X['Description']))

In [169]:
X['Description'][1]

'malgré un scénario moyen ce film  d animation se regarde bien même si la mise en route est difficile et que tout le début soit dans la bande annonce  le film est visuellement très beau également'

In [171]:
X

,Description,key
0,suite de bigfoot junior sorti en revoici la f...,0
1,malgré un scénario moyen ce film d animation ...,1
2,un debut laborieux qui se veut dans lair du te...,2
3,film vu au cinéma tout dabord les graphismes...,3
4,nous avons adoré avec mon fils je trouve que ...,4
5,vu en famille dont un enfant de ans nous av...,5
6,un film très sympa agréable et le mieux à voi...,6


### Retire les mots inutiles

avec NLTK https://www.datacorner.fr/du-nlp-avec-python-nltk/

In [172]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.snowball import FrenchStemmer

french_stopwords = set(stopwords.words('french'))
filtre_stopfr =  lambda text: [token for token in text if token.lower() not in french_stopwords]

In [173]:
X1 = filtre_stopfr(word_tokenize(X['Description'][1]) )
X1

['malgré',
 'scénario',
 'moyen',
 'film',
 'animation',
 'regarde',
 'bien',
 'si',
 'mise',
 'route',
 'difficile',
 'tout',
 'début',
 'bande',
 'annonce',
 'film',
 'visuellement',
 'très',
 'beau',
 'également']

In [174]:
' '.join(X1)

'malgré scénario moyen film animation regarde bien si mise route difficile tout début bande annonce film visuellement très beau également'

In [175]:
X['Description'] = [' '.join(filtre_stopfr(word_tokenize(item))) for item in X['Description']]
X

,Description,key
0,suite bigfoot junior sorti revoici famille big...,0
1,malgré scénario moyen film animation regarde b...,1
2,debut laborieux veut lair temps savère pauvre ...,2
3,film vu cinéma tout dabord graphismes agréable...,3
4,adoré fils trouve cest film quil faudrait mont...,4
5,vu famille dont enfant ans passé très bon mome...,5
6,film très sympa agréable mieux voir conseille ...,6


In [176]:
X['Description'][1]

'malgré scénario moyen film animation regarde bien si mise route difficile tout début bande annonce film visuellement très beau également'

### Stemmatisation / racinistation

In [177]:
stemmer = FrenchStemmer()
X['Description'] = [stemmer.stem(w) for w in X['Description']]

In [181]:
X['Description']

0    suite bigfoot junior sorti revoici famille big...
1    malgré scénario moyen film animation regarde b...
2    debut laborieux veut lair temps savère pauvre ...
3    film vu cinéma tout dabord graphismes agréable...
4    adoré fils trouve cest film quil faudrait mont...
5    vu famille dont enfant ans passé très bon mome...
6    film très sympa agréable mieux voir conseille ...
Name: Description, dtype: object

In [179]:
X['Description'][1]

'malgré scénario moyen film animation regarde bien si mise route difficile tout début bande annonce film visuellement très beau égal'

# Fonction de préparation

In [193]:
REMPLACE_SANS_ESPACE = re.compile("[;:!\'?,\"()\[\]]")
REMPLACE_AVEC_ESPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|[.]")
PUR_NOMBRE = re.compile("[0-9]")

def setClassBin(i):
    if (float(i.replace(',', '.')) > 3):
        return 1
    else:
        return 0
    
def preprocess_reviews(reviews):
    reviews = [PUR_NOMBRE.sub("", line.lower()) for line in reviews] # retire les nomre (comme les années)
    reviews = [line.replace('\n', ' ')  for line in reviews] # Retire les \n (retours chariots)
    reviews = [REMPLACE_SANS_ESPACE.sub("", line.lower()) for line in reviews]
    reviews = [REMPLACE_AVEC_ESPACE.sub(" ", line) for line in reviews]
    return reviews

def prepare_dataset(X):
    X['Description'] = pd.DataFrame(preprocess_reviews(X['Description']))
    french_stopwords = set(stopwords.words('french'))
    filtre_stopfr =  lambda text: [token for token in text if token.lower() not in french_stopwords]
    X['Description'] = [' '.join(filtre_stopfr(word_tokenize(item))) for item in X['Description']]
    stemmer = FrenchStemmer()
    X['Description'] = [stemmer.stem(w) for w in X['Description']]
    
    yList = [setClassBin(x) for x in X.Note]
    y = pd.DataFrame(yList)
    
    X = src.drop('Note', axis=1)
    return X, y

In [194]:
XTrain, yTrain = prepare_dataset(src.copy())

In [195]:
XTrain

,Description,key
0,"Suite de ""Bigfoot Junior"" sorti en 2017, revoi...",0
1,malgré un scénario moyen ce film d animation ...,1
2,Un debut laborieux qui se veut dans l'air du t...,2
3,"Film vu au cinéma. \nTout d'abord, les graphis...",3
4,Nous avons adoré avec mon fils. Je trouve que ...,4
5,"Vu en famille, dont un enfant de 4 ans 1/2, no...",5
6,"un film très sympa ,agréable et le mieux à vo...",6


In [196]:
yTrain

,0
0,0
1,0
2,0
3,1
4,1
5,1
6,1
